In [34]:
data_csv = '''
page_protocol,page_host,api_protocol,api_host,api_path,api_method,psm,scope,field_name,field_path,field_sample,data_type_name
https,market-au.example.com,https,market-au.example.com,/api/v1/product/prohibited/words/check,post,oec.product.product_api,query,browser_language,browser_language,en-US,Language
https,market-au.example.com,https,market-au.example.com,/api/v1/product/prohibited/words/check,post,oec.product.product_api,query,browser_language,browser_language,en-US,Language
https,market-au.example.com,https,market-au.example.com,/api/v1/product/prohibited/words/check,post,oec.product.product_api,query,browser_language,browser_language,en-US,Language
https,market-au.example.com,https,open-api.example.com,/v0/oauth/check_qr,get,abcd.openapi.gateway,query,token,token,abcd_aueast3,Account Setting
https,market-au.example.com,https,open-api.example.com,/v0/oauth/check_qr,get,abcd.openapi.gateway,query,token,token,wxyz_aueast3,Account Setting
https,market-au.example.com,https,market-au.example.com,/api/v1/product/list/seller/warehouses,get,oec.product.product_api,query,browser_version,browser_version,5.0%20%28Windows%20NT%2010.0%3B%20Win64%3B%20x64%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F120.0.0.0%20Safari%2F537.36,User Agent
https,market-au.example.com,https,market-au.example.com,/api/v1/seller/holiday_mode/list,get,oec.seller.profile_api,query,locale,locale,en,Language
'''

In [35]:
import pandas as pd
from io import StringIO

data = pd.read_csv(StringIO(data_csv))
target_column = 'data_type_name'
feature_columns = data.columns[data.columns != target_column]
# target_columns = data.columns[data.columns == target_column]  # In case multiple targets

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   page_protocol   7 non-null      object
 1   page_host       7 non-null      object
 2   api_protocol    7 non-null      object
 3   api_host        7 non-null      object
 4   api_path        7 non-null      object
 5   api_method      7 non-null      object
 6   psm             7 non-null      object
 7   scope           7 non-null      object
 8   field_name      7 non-null      object
 9   field_path      7 non-null      object
 10  field_sample    7 non-null      object
 11  data_type_name  7 non-null      object
dtypes: object(12)
memory usage: 804.0+ bytes


# Step 1. Collect some info

TODO: Remove NaN

In [88]:
# Collect the statistics for categorical and numerical columns in X
categorical = data[feature_columns].select_dtypes(['category', 'object']).columns

# categorical_unique_values = [data[col].unique() for col in categorical]
# categorical_num_unique_values = [len(vals) for vals in categorical_unique_values]

# Collect statistics for y
if data[target_column].dtype in ('object', 'category'):
    # Classification
    target_unique_values = data[target_column].unique()
    output_dimensions = len(target_unique_values)
    if output_dimensions == 2:
        output_dimensions = 1  # Binary classification can be done with a single output
else:
    # Regression
    output_dimensions = 1

data[categorical] = data[categorical].astype('category')

In [89]:
# Create the X, y data
X = data[feature_columns]
y = data[target_column]

X[categorical] = X[categorical].apply(lambda column: column.cat.codes)

if 

data[categorical[3]].cat.categories

/tmp/ipykernel_2054799/944709309.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[categorical] = X[categorical].apply(lambda column: column.cat.codes)


Index(['market-au.example.com', 'open-api.example.com'], dtype='object')

In [82]:
X

,page_protocol,page_host,api_protocol,api_host,api_path,api_method,psm,scope,field_name,field_path,field_sample
0,0,0,0,0,1,1,1,0,0,0,3
1,0,0,0,0,1,1,1,0,0,0,3
2,0,0,0,0,1,1,1,0,0,0,3
3,0,0,0,1,3,0,0,0,3,3,1
4,0,0,0,1,3,0,0,0,3,3,4
5,0,0,0,0,0,0,1,0,1,1,0
6,0,0,0,0,2,0,2,0,2,2,2


# Step 2. Convert the data to numerical values

In [56]:
feature_tokens = []
for col_idx, col in enumerate(categorical):
    string_to_int = {val: idx for idx, val in enumerate(categorical_unique_values[col_idx])}
    int_to_string = {idx: val for val, idx in string_to_int.items()}
    all_tokens = {**string_to_int, **int_to_string}
    feature_tokens.append(all_tokens)

# Step 3. Create model

In [48]:
from torch import nn

from tab_transformer_pytorch import TabTransformer

# Hyperparameters from the original paper
params = {
    'dim': 32,
    'depth': 6,
    'heads': 8,
    'attn_dropout': 0.1,
    'ff_dropout': 0.1,
    'mlp_hidden_mults': (4, 2),
    'mlp_act': nn.ReLU(inplace=True),  # Can be reused, as this is stateless
}


model = TabTransformer(
    categories = categorical_num_unique_values,
    num_continuous = len(numerical),
    dim_out = output_dimensions,
    # continuous_mean_std = mean_std,
    **params,
)

In [49]:
model

TabTransformer(
  (transformer): Transformer(
    (embeds): Embedding(30, 32)
    (layers): ModuleList(
      (0-5): 6 x ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (to_qkv): Linear(in_features=32, out_features=384, bias=False)
            (to_out): Linear(in_features=128, out_features=32, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=32, out_features=256, bias=True)
              (1): GEGLU()
              (2): Dropout(p=0.1, inplace=False)
              (3): Linear(in_features=128, out_features=32, bias=True)
            )
          )
        )
      )
    )
  )
  (mlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=352, out_features=176, bias=Tru